<a href="https://colab.research.google.com/github/himanshuchrist/LLM/blob/main/Himanshu_520_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Load the IMDb Dataset

In [14]:
from datasets import load_dataset
dataset=load_dataset('imdb')

Take equal ratio of positive and negative reviews

In [59]:
import random
postive = [x for x in dataset['train'] if x['label'] == 1][:2500]
negative = [x for x in dataset['train'] if x['label'] == 0][:2500]
train_dataset = postive + negative
random.shuffle(train_dataset)
positive_test = [x for x in dataset['test'] if x['label'] == 1][:2500]
negative_test = [x for x in dataset['test'] if x['label'] == 0][:2500]
test_dataset = positive_test + negative_test
random.shuffle(test_dataset)

Loading Tokenizer and BERT model

In [16]:
from transformers import BertTokenizer, TFBertForSequenceClassification
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
model=TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Defining Train and Test data

In [62]:
train_texts = [x['text'] for x in train_dataset]
train_labels = [x['label'] for x in train_dataset]
test_texts = [x['text'] for x in test_dataset]
test_labels = [x['label'] for x in test_dataset]

Tokenizing train and test test

In [63]:
import tensorflow as tf
train_encodings=tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings=tokenizer(test_texts, truncation=True, padding=True, max_length=128)

Mapping train and test to labels

In [64]:
train_dataset=tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset=tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))
train_dataset=train_dataset.shuffle(1000).batch(8).prefetch(tf.data.AUTOTUNE)
test_dataset=test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

Compiling the model

In [65]:
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Fitting the model

In [66]:
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3
625/625 [==============================] - 267s 331ms/step - loss: 0.5661 - accuracy: 0.6984 - val_loss: 0.3398 - val_accuracy: 0.8564
Epoch 2/3
625/625 [==============================] - 198s 317ms/step - loss: 0.2577 - accuracy: 0.8938 - val_loss: 0.3479 - val_accuracy: 0.8642
Epoch 3/3
625/625 [==============================] - 200s 320ms/step - loss: 0.1160 - accuracy: 0.9582 - val_loss: 0.4610 - val_accuracy: 0.8564


Checking the Accuracy

In [67]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.2f}")

625/625 [==============================] - 51s 82ms/step - loss: 0.4610 - accuracy: 0.8564
Test Accuracy: 0.86


Sentiment Prediction function

In [68]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    outputs = model(inputs)
    logits = outputs.logits
    predictions = tf.nn.softmax(logits, axis=-1)
    sentiment = tf.argmax(predictions, axis=1).numpy()[0]
    return sentiment

Predicting

In [73]:
text = input("Enter text for sentiment analysis: ")
sentiment = predict_sentiment(text)
sentiment_label = "Positive" if sentiment == 1 else "Negative"
print(f"Sentiment: {sentiment_label}")

Enter text for sentiment analysis: this is a good example of bad movie
Sentiment: Negative
